In [ ]:
import sys
sys.path.append('../src')

import yaml
from src.data.data_loader import load_data
from src.data.prepare_data import prepare_data
import pandas as pd
import mlflow
from src.models.model import modeling

# start mlflow ui
import subprocess, time
ui_process = subprocess.Popen(["python", "-m", "mlflow", "ui"])
time.sleep(5)
ui_process.terminate()

In [ ]:
# read config
with open('main/config_inference.yml', 'r') as file:
    config=yaml.load(file, Loader=yaml.SafeLoader)

In [ ]:
# load and prepare data
dataframes=load_data(config=config['data_loader'])
dataframes=prepare_data(dataframes=dataframes)

In [ ]:
# mlflow tracking
mlflow.set_tracking_uri(uri=config['tracking']['uri'])
mlflow.set_experiment(experiment_name=config['tracking']['experiment_name'])
mlflow.autolog(disable=True)

with mlflow.start_run(run_name=config['tracking']['run_name']):
    with mlflow.start_run(run_name=config['model']['algo_class'], nested=True):
        # log multiple inputs
        mlflow.set_tag(key='algorithm', value=config['model']['algo_class'])
        mlflow.log_params(params=config['model']['params'])
        mlflow.log_dict(dictionary=config, artifact_file="config_inference.yml")

        # test best algortihm
        model_evaluation=modeling(
            algo_class=config['model']['algo_class']
            , params=config['model']['params']
            )
        model_evaluation.fit(train_set=dataframes['ratings'])
        pred=model_evaluation.inference()
        
        # log model
        mlflow.sklearn.log_model(sk_model=model_evaluation.algo_class, artifact_path=config['model']['algo_class']+'_model')